Notebook to show how many spin flips it takes to come to equilibrium on the Pokec network. Also use this to explore the impact of different initial conditions/parameters on the burn-in time.

To do:
- Change convention for showing the number of timesteps from $T$ to $T$/$N$ (i.e. number of spin flips per node).

Created on 06/01/20

In [1]:
import networkx as nx
import numpy as np
import pandas as pd
import ast
import seaborn as sns
import matplotlib.pyplot as plt
import tqdm
import random
import itertools
import matplotlib

from ising_block_level_influence import N_Block_sbm_class as NBlock
from ising_block_level_influence import projection_simplex as proj
from ising_block_level_influence import mean_field_IIM
from spatial_spin_monte_carlo import spatial_spin_monte_carlo as Spins
import Pokec_processing as  PokProc

from pokec_utils import *

### Read in processed data

This data has been generated using:

1. **make_Pokec_reduced_profiles.ipynb** - which then feeds data into:

2. The pre-processing script **'make_bratislava_graph_and_blocks.py'**.

In [2]:
graph = nx.read_graphml('Data/Bratislava_graph.graphml')
bratislava_profiles = pd.read_csv("Data/bratislava_profiles.csv")
coupling_graph = nx.read_graphml('Data/Bratislava_coupling.graphml')
block_data = pd.read_csv('Data/block_info.csv',converters={'Block' : ast.literal_eval})
mean_block_ages = list(block_data['average_age'])
block_sizes = list(block_data['block_size'])
block_names = list(block_data['block_name'])
block_data['age_group'] = [ 'ages_' + k.split('_')[-1] for k in list(block_data['block_name'])]
block_data.head()

,Unnamed: 0,block_name,block_size,average_age,age_group
0,0,Okolie_ages_1-17,1234,9.0,ages_1-17
1,1,Okolie_ages_18-21,1939,19.5,ages_18-21
2,2,Okolie_ages_22-28,3154,25.0,ages_22-28
3,3,Okolie_ages_29-112,3458,70.5,ages_29-112
4,4,Petrzalka_ages_1-17,1398,9.0,ages_1-17


In [3]:
def linear_field(x : np.ndarray,gradient :float) :
    return gradient*x

#Scale ages to [-1,1]:
rescaled_ages = [ (k-np.mean(mean_block_ages))/(max(mean_block_ages)-min(mean_block_ages)) for k in mean_block_ages ]

beta_c = Spins.crit_beta_sparse(graph)

bratislava_profiles_indices = bratislava_profiles.reset_index()
groups = [ bratislava_profiles_indices.loc[bratislava_profiles_indices['block']==block] for block in block_names]
groups_node_ids = [list(k['index']) for k in groups]

Computing critical temperature


## Spin series: sample chains for different initial conditions

Use the parameters used in Figure 3 of main text.

This version generates spin series for a single metastable state.

In [4]:
#Seed the random number generators:
seed = 1
random.seed(seed)
np.random.seed(seed)

beta_factor_vals = [(10**k) for k in np.linspace(-1,1.8,25)]
grad_vals = [1.0,5.0,10.0]

beta_f=8.0
gradient=10.0

T = int(3*(10**6))
T_Burn = 0 # Set this as zero to return the full length in 'Run_MonteCarlo_Average'

#Field setup:

age_field = [linear_field(a,gradient) for a in rescaled_ages ]
age_field_map = {k:j for k,j in zip(list(block_data['age_group']),age_field)}
background_field = np.asarray([age_field_map[k] for k in list(bratislava_profiles['age_group'])])

# Inverse temperature: 
beta = beta_f*beta_c

relab_graph = nx.relabel.convert_node_labels_to_integers(graph)


samples=50

sample_spacing=1000


# init type can be 'pos','neg','random'
init_type='pos'
for init_type in ['pos'] : # need to re-declare the dataframe!!
    mc_chains={}
    index=0
    mc_chains['timestep']=list(np.arange(0,T,1))[0::sample_spacing]
    for samp in tqdm.tqdm_notebook( range(samples) ) :

        # MC simulations: initial condition
        if init_type=='neg' :
            initial_state = -1.0*np.ones(len(graph))
        elif init_type=='random' :
            initial_state = np.random.choice([-1,1],len(graph))
        elif init_type=='pos' :
            initial_state = 1.0*np.ones(len(graph))


        spin_series = Run_MonteCarlo_Average(relab_graph, T, beta_f,beta_c, T_Burn=T_Burn,addition_control=None,sampling_method="Metropolis",full_graph_field=background_field,initial_state=initial_state)

        mc_chains[f"chain_{index}"]=spin_series[0::sample_spacing]

        # Add params to filename or dataset.
        df=pd.DataFrame(mc_chains)
        df.to_csv(f"Data/mc_chain_examples/{init_type}_init_chain_data_beta_{beta_f}_g_{gradient}".replace(".","-") + ".csv",index=False)

        index+=1

In [5]:
data=pd.read_csv(f"Data/mc_chain_examples/random_init_chain_data_beta_{beta_f}_g_{gradient}".replace(".","-") + ".csv")
data

,timestep,chain_0,chain_1,chain_2,chain_3,chain_4,chain_5,chain_6,chain_7,chain_8,...,chain_73,chain_74,chain_75,chain_76,chain_77,chain_78,chain_79,chain_80,chain_81,chain_82
0,0,-0.000811,0.006288,-0.003718,0.004462,-0.002096,-0.001082,0.001149,0.003786,0.008181,...,-0.006828,-0.010615,-0.008181,0.003178,-0.008654,-0.002840,0.002704,-0.002907,-0.009060,0.012034
1,1000,0.001825,0.009600,-0.003110,0.005138,-0.002231,0.000135,0.003989,0.003313,0.005612,...,-0.002975,-0.009938,-0.008248,0.002840,-0.003854,-0.001893,0.002975,0.002975,-0.005409,0.011832
2,2000,0.001825,0.009871,-0.003718,0.008316,-0.003042,0.000473,0.003651,0.004665,0.003921,...,-0.002772,-0.007302,-0.004733,0.003448,-0.000947,-0.002096,0.006490,0.004259,-0.005341,0.013454
3,3000,0.003989,0.011832,-0.001149,0.009262,-0.003042,0.005071,0.006828,0.005003,0.006288,...,-0.000947,-0.008586,-0.003921,0.002502,0.003313,-0.000406,0.007099,0.006626,-0.005071,0.013387
4,4000,0.006288,0.017037,0.000000,0.008316,-0.000676,0.004597,0.011291,0.010074,0.008383,...,0.000744,-0.007302,-0.004395,0.002975,0.002975,-0.000203,0.011832,0.008451,-0.004597,0.013927
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,2995000,-0.168008,-0.170374,-0.175377,-0.171929,-0.174295,-0.182949,-0.174566,-0.174025,-0.175445,...,-0.173822,-0.176932,-0.174768,-0.174430,-0.171320,-0.174768,-0.174363,-0.170374,-0.173957,-0.170847
2996,2996000,-0.167737,-0.170239,-0.174904,-0.171726,-0.173349,-0.180583,-0.174971,-0.174160,-0.174633,...,-0.173011,-0.176932,-0.175783,-0.175985,-0.171861,-0.174228,-0.174295,-0.169833,-0.173146,-0.171185
2997,2997000,-0.166993,-0.170036,-0.172808,-0.171658,-0.172875,-0.180110,-0.174025,-0.173551,-0.174092,...,-0.173687,-0.176053,-0.175850,-0.175512,-0.171591,-0.174566,-0.175174,-0.169563,-0.172267,-0.171658
2998,2998000,-0.165100,-0.171726,-0.172064,-0.170915,-0.171523,-0.179907,-0.172740,-0.173890,-0.174971,...,-0.173281,-0.176323,-0.176594,-0.175039,-0.171929,-0.175309,-0.176188,-0.169765,-0.172808,-0.172402
